In [6]:
import os
current_dir = os.getcwd()
if "tscc" in current_dir:
    os.chdir("/tscc/nfs/home/bax001/scratch/CSE_251B")

In [1]:
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

## LOAD INPUT DATA

In [9]:
train = np.load("data/train.npz")

In [13]:
train_data = train["data"]

## DATASET and DATALOADER

In [7]:
class AgentTrajectoryDataset(Dataset):
    def __init__(self, npz_file_path):
        """
        Args:
            npz_file_path (str): Path to .npz file.
        """
        super().__init__()
        data = np.load(npz_file_path)
        self.data = data['data']  # (N, 50, T, 6)
        
    def __len__(self):
        return self.data.shape[0]
    
    def __getitem__(self, idx):
        sample = self.data[idx]  # (50, T, 6)
        
        position = sample[..., 0:2]    # (50, T, 2)
        velocity = sample[..., 2:4]    # (50, T, 2)
        heading = sample[..., 4]       # (50, T)
        object_type = sample[..., 5]   # (50, T)

        return {
            'position': torch.tensor(position, dtype=torch.float32),
            'velocity': torch.tensor(velocity, dtype=torch.float32),
            'heading': torch.tensor(heading, dtype=torch.float32),
            'object_type': torch.tensor(object_type, dtype=torch.long)
        }

# Instantiate datasets
train_dataset = AgentTrajectoryDataset('data/train.npz')
test_dataset = AgentTrajectoryDataset('data/test_input.npz')

# Create dataloaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Example: peek at one batch
batch = next(iter(train_loader))
print(batch['position'].shape)     # (64, 50, 110, 2)
print(batch['velocity'].shape)     # (64, 50, 110, 2)
print(batch['heading'].shape)      # (64, 50, 110)
print(batch['object_type'].shape)  # (64, 50, 110)

torch.Size([64, 50, 110, 2])
torch.Size([64, 50, 110, 2])
torch.Size([64, 50, 110])
torch.Size([64, 50, 110])


## Train Val Test Split

In [14]:
from torch.utils.data import random_split

# 80% train, 20% val split
train_size = int(0.8 * len(train_dataset))
val_size = len(train_dataset) - train_size

train_dataset, val_dataset = random_split(train_dataset, [train_size, val_size])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

print(f"Train samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Test samples: {len(test_dataset)}")


Train samples: 8000
Validation samples: 2000
Test samples: 2100


## MODEL DEF

In [ ]:
# --- Model Definition Section ---

import torch.nn as nn

class AgentMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(AgentMLP, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )
        
    def forward(self, x):
        return self.model(x)



AgentMLP(
  (model): Sequential(
    (0): Linear(in_features=11000, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1, bias=True)
  )
)


## Training loop

In [17]:
# --- Training Loop Section ---

import torch.optim as optim

def train_model(model, train_loader, val_loader, num_epochs=50, lr=1e-3, patience=5, device='cuda'):
    """
    Train the model with early stopping on validation loss.
    """
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()  # Placeholder loss; change depending on your task
    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for batch in train_loader:
            inputs = batch['position'].to(device)  # (batch_size, 50, 110, 2)
            inputs = inputs.view(inputs.size(0), -1)  # Flatten for MLP

            targets = torch.zeros(inputs.size(0), 1, device=device)  # Dummy target, replace with your labels

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        train_loss /= len(train_loader)

        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for batch in val_loader:
                inputs = batch['position'].to(device)
                inputs = inputs.view(inputs.size(0), -1)

                targets = torch.zeros(inputs.size(0), 1, device=device)

                outputs = model(inputs)
                loss = criterion(outputs, targets)

                val_loss += loss.item()

        val_loss /= len(val_loader)

        print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Val Loss = {val_loss:.4f}")

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            patience_counter += 1

        if patience_counter >= patience:
            print(f"Early stopping triggered at epoch {epoch+1}")
            break

    print("Training complete. Best validation loss:", best_val_loss)


In [18]:
# Instantiate model
model = AgentMLP(input_dim=50*110*2, hidden_dim=512, output_dim=1)

# Train the model
train_model(model, train_loader, val_loader, num_epochs=50, lr=1e-3, patience=5, device='cuda')


Epoch 1: Train Loss = 10184459.7407, Val Loss = 58468.0651
Epoch 2: Train Loss = 64713.4666, Val Loss = 148652.4160
Epoch 3: Train Loss = 32536.1037, Val Loss = 8754.0025
Epoch 4: Train Loss = 20561.2397, Val Loss = 25131.8058
Epoch 5: Train Loss = 55391.8898, Val Loss = 4426.9980
Epoch 6: Train Loss = 10749.5299, Val Loss = 8635.6568
Epoch 7: Train Loss = 9259.2980, Val Loss = 3356.3224
Epoch 8: Train Loss = 10858.4170, Val Loss = 3558.3267
Epoch 9: Train Loss = 12638.1812, Val Loss = 9750.5086
Epoch 10: Train Loss = 26469.0073, Val Loss = 53657.9272
Epoch 11: Train Loss = 156024.7683, Val Loss = 21455.5379
Epoch 12: Train Loss = 28435.9064, Val Loss = 7792.7537
Early stopping triggered at epoch 12
Training complete. Best validation loss: 3356.3223991394043


## Run on test set

## Run on external test set and submit to Kaggle